In [ ]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime

In [ ]:
### To save from multiple print and display statements, below import
### will let you see multiple outputs in same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df = pd.read_csv("../input/german-credit-data-with-risk/german_credit_data.csv", index_col=0)

### below renaming is just to make the data consistent with the one on my local
df.rename(columns = {'Checking account': 'Credit History', 'Sex': 'Gender'}, inplace=True)

### Preliminary data analysis

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
### check NaNs
df.isnull().sum()

In [ ]:
feature = 'Gender'
df[feature].value_counts()

In [ ]:
df[feature].unique()

In [ ]:
df.nunique()

In [ ]:
### show all object type columns
df.select_dtypes(include=['object']).columns

In [ ]:
### shallow vs deep copy of a dataframe
### In deep copy, a new df will be created where any changes to the new df, df_deep will not be reflected in original df
df_deep=df.copy(deep=True)
df_shallow = df.copy(deep=False)

### datetime conversion

In [ ]:
### Let's create a date column
### Below generates the date for 1000 periods on daily frequency
df['date'] = pd.date_range('1/1/2000', periods=1000)

In [ ]:
df['date'].dtype

In [ ]:
df['date'][0]

In [ ]:
### As an example, if we want to create a date range at hourly frequency for 5 periods
pd.date_range('1/1/2000', freq='H', periods=5)

In [ ]:
### Passing errors='ignore' will return the input date if the date does not meet the timestamp limitations
### As per https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-timestamp-limits, 
### the timestamp can be represented using 64 bit in following range: 
print("Minimum date:", pd.Timestamp.min)
print("Maximum date:", pd.Timestamp.max)

In [ ]:
### Input date outside the timestamp limitation is outputted as it is, when errors = 'ignore' was passed
pd.to_datetime('2265-04-11', format='%Y-%m-%d', errors = 'ignore')

In [ ]:
### Input date outside the timestamp limitation is marked as NaT, when errors = 'coerce' was passed
pd.to_datetime('2265-04-11', format='%Y-%m-%d', errors = 'coerce')

In [ ]:
### takes MM/DD/YY format
dateutil.parser.parse('1/1/2000')

In [ ]:
### for cases like below, specify the parameter dayfirst as True
dateutil.parser.parse('10/1/2000', dayfirst=True)

In [ ]:
### As against default
dateutil.parser.parse('10/1/2000')

In [ ]:
### But for case like below with ambiguous date, passing dayfirst is ignored and 13 is assumed as date
dateutil.parser.parse('10/13/2000', dayfirst=True)

In [ ]:
### convert a string to datetime
datetime.datetime.strptime('18-1-2000','%d-%m-%Y')

In [ ]:
### https://strftime.org/
today =datetime.datetime.today()
print("today's date:", today)
today.strftime("%d %B %Y")

In [ ]:
### chnaging B (Full month name) to b(abbreviated month name) and Y(in century) to y(without century)
today.strftime("%d %b %y")

### Key operations

In [ ]:
df.sort_values(by=['Age'])

In [ ]:
### setting index
index_col ='date'
df = df.set_index(index_col)

In [ ]:
df

In [ ]:
### Now, lets shift the row by 1 
df.shift(1)

In [ ]:
### Observe the difference by specifying the frequency. Here, the row corresponding to 2000-01-01
### gets eliminated and data shifts by 1 day, creating the observation at 2002-09-27
df.shift(1, freq="D")

In [ ]:
df.shift(-1)

In [ ]:
### filter based on index. Lets index the data based on Age and Duration
df_age_duration = df.set_index(['Age', 'Duration'])
df_age_duration.head()

In [ ]:
### get level 0 index values, which corresponding to Age and filter the dataframe for Age > 30
df_age_duration[df_age_duration.index.get_level_values(0) > 30]

In [ ]:
### dropping wrt rows and columns
### how = 'all' drops the rows or columns which have all NaNs

In [ ]:
### if any value in a row is NA, then it is dropped
df.dropna(axis = 0, how='any')

In [ ]:
### if any value in a column are NA, then it is dropped
df.dropna(axis = 1, how='any')

In [ ]:
### lowercase, uppercase, capitalize the column names
[k.lower() for k in list(df.columns)]
[k.upper() for k in list(df.columns)]
[k.capitalize() for k in list(df.columns)]

### concatenating rows and columns

In [ ]:
df_dup = pd.concat([df, df], axis=0)
df_dup

In [ ]:
### column wise concatenation
df_age = df[['Age']]
pd.concat([df, df_age], axis=1)

### Remove duplicates 

In [ ]:
df_dup.drop_duplicates()

### Now, let's groupby and count the instances per 'Gender'

In [ ]:
feature = ['Gender']
df.groupby('Gender').size()

In [ ]:
### multiple functions: mean, transform, var, nth(-10), nlargest etc
df.groupby(feature).mean()
df.groupby(feature).std()

In [ ]:
df.groupby(feature).first()
df.groupby(feature).last()

In [ ]:
df.groupby('Gender').apply(lambda x: x.nlargest(2, 'Credit amount')).reset_index(drop=True)  

In [ ]:
### apply function
df.groupby(['Gender'])['Risk'].apply(np.size)

In [ ]:
### selecting, dropping, renaming columns

In [ ]:
df_dtypes = pd.DataFrame((df.dtypes == 'object'), columns = ['obj_type'])
selected_cols = list(df_dtypes[(df_dtypes.obj_type == True)].index)
df[selected_cols]

In [ ]:
df.columns
df.rename(columns = {'Saving accounts': 'richness_quotient'}).columns

In [ ]:
drop_cols = ['Saving accounts']
df.drop(columns = drop_cols)

In [ ]:
feature = 'Age'
df_subset = df[df[feature] >= 30]
np.sort(df_subset['Age'].unique())

In [ ]:
feature = 'Housing'
df[df[feature] == 'own'][feature].unique()

In [ ]:
feature = 'Purpose'
des_values = ['car', 'education']
df[df[feature].isin(des_values)][feature].unique()

In [ ]:
feature = 'Saving accounts'
df[~df[feature].isna()]

In [ ]:
### np.where vs np.select
### np.where is used where 1 or 2 values are returned, but in case of multiple values , np.select is used

In [ ]:
criteria = [df['Age'] <30, (df['Age']<45)&(df['Age']>=30), df['Age']>=45]
assign_vals = [1,2,3]
df['Age_binned'] = np.select(criteria, assign_vals)

In [ ]:
df.head()

In [ ]:
### pivot_table, default agg_func is 'mean'
df.pivot_table(columns='Housing', values='Credit amount')